# Post

In [7]:
import requests

server_url = ['61.74.232.165:5000','192.168.0.198:5000', 'localhost:5000'][1]
upload_url=server_url + '/upload'

# 키 전달
def set_key():
    # 미리 정해져있음 
    flists = [("enc_key", 'EncKey.txt'), ('mul_key', 'MulKey.txt')]
    for dtype, fn in flists:
        r = requests.post('http://'+upload_url, 
                          files={'file':open(fn, 'rb')}, 
                          headers={'dtype':dtype})
        print(r)
        if not r.ok:
            print("ERROR")
            return -1
    return True

# 암호문 전달
def send_ctxt(action, ori):
    """e.g., ctx_05_e.dat"""
    files = {'file': open(f'./ctx_{action.zfill(2)}_{ori}.dat','rb')}
    header = {'dtype':"ctxt", "action":f'{action}'}
    r = requests.post('http://'+upload_url, files=files, headers=header)
    return r.ok

with open("EncKey.txt", "w") as f:
    f.write("EEEE")
    
with open("MulKey.txt", "w") as f:
    f.write("MMM")

r = set_key()
print(r)

<Response [200]>
<Response [200]>
True


In [40]:
r = send_ctxt('13', 'a')

ConnectionError: HTTPConnectionPool(host='61.74.232.165', port=5000): Max retries exceeded with url: /upload (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7feef733ae50>: Failed to establish a new connection: [Errno 111] Connection refused'))

# get

서버에 계산된 암호문 파일 받은 후 동형암호 라이브러리로 읽어서 복호화, 결과 확인.
`
**서버 계산이 완료될 때 까지 5초에 한번씩 get 시도**


In [21]:
from time import sleep

def save_binary(r, fn_save):
    if r.status_code == 200:
        with open(fn_save, 'wb') as f:
            for chunk in r:
                f.write(chunk)
    else:
        raise FileNotFoundError

result_url='http://'+server_url + '/result'

def get_result(result_url):
    n_try = 0
    while n_try < 10:
        r = requests.get(result_url, stream=True)
        if r.status_code == 200:
            save_binary(r,'./client_results/pred0.dat')
            print("Result recieved.")
            break
        else:
            sleep(5)
            n_try+=1
    else:
        print("Retry limit reached. Try again later")
        return False
    return r

In [11]:
ls './client_results'

In [24]:
r = get_result(result_url)

Result recieved.


### 결과 확인 예시

In [ ]:
# HEAAN 라이브러리로 ctxt 파일 로드
logp = 30
logq = 300
nslots = 2**14
ctx_pred = he.Ciphertext(logp, logq, nslots)
he.SerializationUtils.readCiphertext(ctx_pred, fn_ctx)
# decryptor로 decrypt
dec=decrypt(self.scheme, self.secretKey, ctx_pred, self.parms)
# 결과 출력 
print(np.argmax(dec))